### Модели ходьбы. Capture step


#### Linear Inverted Pendulum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

class LIP:
    def __init__(self, m = 1.0, l = 1.0, g = 10.0,
                 x_contact0 = 0.2, x0 = 0.3, xd0 = -0.1,
                 dt = 0.01, WIND_X = 700, scale = 100):
        self.m = m
        self.l = l
        self.g = g
        
        self.x = np.array([[x0], [xd0]])
        self.dt = dt
                        
        self.WIND_X = WIND_X
        self.scale = scale
        
        self.x_contact = x_contact0
    
    def change_x_contact(self, new_x_contact):
        self.x[0, 0] += self.x_contact - new_x_contact
        
        self.x_contact = new_x_contact
    
    def get_state(self):
        return self.x
    
    def propagate_system(self):
        g = self.g
        l = self.l
        
        xdd = g / self.l * self.x[0, 0]
        
        self.x[1, 0] += xdd * self.dt
        self.x[0, 0] += self.x[1, 0] * self.dt
    
    def draw(self, cx, cy, scale = 100, color = (234, 123, 123), canvas = None):
        if (canvas is None):
            canvas = np.ones((700, 700, 3)) * 0

        wind_h, wind_w, _ = canvas.shape

        cv2.line(canvas, (0, wind_h // 2), (wind_w, wind_h // 2), color, 2)

        cv2.circle(canvas, (int(wind_w // 2 + (self.x_contact + self.x[0, 0]) * scale),
                            int(wind_h // 2 - self.l * scale)), 7, color, 2)

        cv2.line(canvas, (int(wind_w // 2 + (self.x_contact + self.x[0, 0]) * scale),
                          int(wind_h // 2 - self.l * scale)),
                         (int(wind_w // 2 + self.x_contact * scale), int(wind_h // 2)), color, 2)

        return canvas

def run_LIP_episode(x0 = 1.2, xd0 = -0.1, scale = 30, dt = 0.05, WIND_X = 700, WIND_Y = 700):
    canvas = np.ones((WIND_Y, WIND_X, 3), np.uint8) * 70
    
    lip = LIP(m = 1.0, l = 1.0, g = 10.0, x_contact0 = 0, x0 = x0, xd0 = xd0,
              dt = dt, WIND_X = WIND_X, scale = scale)

    iter_num = 75000
    i = 1

    x_traj = []
    u_traj = []
    
    cv2.namedWindow("LIP")
    cv2.createTrackbar("target", "LIP", WIND_X // 2, WIND_X, lambda i : i)
    
    while(True):
        state = lip.get_state()

        x_traj.append(state[0, 0])

        if (i % 10 == 0):
            target_pos = (cv2.getTrackbarPos("target", "LIP") - WIND_X // 2) / scale
            
            print("contact", lip.x_contact)
            print("state[0, 0]", state[0, 0])
            print("target", target_pos)
            
            # YOUR CODE BELOW
            
            control = 2 * state[0, 0] + lip.x_contact

            # YOUR CODE ABOVE

            #print(control)
            #print()

            lip.change_x_contact(control)
            u_traj.append(control)
        
        else:
            u_traj.append(0)
        
        lip.propagate_system()
                
        canvas = cv2.addWeighted(canvas, 0.93, canvas, 0, 0)
        
        lip.draw(WIND_X // 2, WIND_Y // 2, scale = scale, canvas = canvas)

        cv2.imshow("LIP", canvas)
        
        i += 1

        if (i > iter_num):
            break
        
        key = cv2.waitKey(400) & 0xFF
        
        if (key == ord('q')):
            break
    
    cv2.destroyAllWindows()
    cv2.waitKey(10)
    
    return x_traj, u_traj

x_hist, u_hist = run_LIP_episode(x0 = 0.4, xd0 = 0.1, scale = 200, WIND_X = 1300, WIND_Y = 600)

contact 0
state[0, 0] 0.9999870880203272
target 0.0
contact 1.5999741760406545
state[0, 0] 0.3237706568910742
target 0.0
contact 2.7292948516474262
state[0, 0] -0.825348231710513
target 0.0
contact 1.8114653231086022
state[0, 0] -0.8800367284310506
target 0.0
contact 0.033804460514800405
state[0, 0] 0.2774834378265074
target 0.0
contact -0.18401859878484716
state[0, 0] 1.096872090412617
target 0.0
contact 1.5188714209797118
state[0, 0] 0.4344177684987221
target 0.0
contact 2.866997225845483
state[0, 0] -0.853083883948324
target 0.0


: 

#### Capture step

via grid search